In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

In [3]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)

#base_url = 'https://www.metacritic.com/browse/game/pc/all/all-time/userscore/?releaseYearMin=1958&releaseYearMax=2025&platform=pc&page=1'
#url = base_url
#driver.get(url)
#driver.maximize_window()

soup = BeautifulSoup(driver.page_source, 'html.parser')
title_nodes = soup.select('div.c-finderProductCard_title[data-title]')
titles = [n['data-title'].strip() for n in title_nodes]
#data_rows = soup.find_all('div', class_='c-finderProductCard_title') #find_all은 리스트로 가져옴
# data_rows = parent.find_all('h3', class_='c-finderProductCard_titleHeading')
BASE = ("https://www.metacritic.com/browse/game/pc/all/all-time/userscore/"
        "?releaseYearMin=1958&releaseYearMax=2025&platform=pc&page={page}")

all_names = []

for page in range(1,76):  # 1 ~ 75
    url = BASE.format(page=page)
    driver.get(url)
    driver.maximize_window()

    # 목록 로드 대기 (타이틀 div가 뜰 때까지)
    WebDriverWait(driver, 12).until(
        EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, 'div.c-finderProductCard_title[data-title]')
        )
    )

    # data-title 속성만 추출하면 번호 없이 깔끔
    elems = driver.find_elements(By.CSS_SELECTOR, 'div.c-finderProductCard_title[data-title]')
    names = [e.get_attribute('data-title').strip() for e in elems]
    all_names.extend(names)

    print(f"page {page}: {len(names)} items (total {len(all_names)})")
    time.sleep(0.5)  # 서버에 예의 + 로딩 여유

# 저장(선택)
pd.DataFrame({'title': all_names}).to_csv('metacritic_pc_userscore_green.csv', index=False)
print("done:", len(all_names))

The chromedriver version (138.0.7204.94) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (139.0.7258.67); currently, chromedriver 139.0.7258.68 is recommended for chrome 139.*, so it is advised to delete the driver in PATH and retry


page 1: 24 items (total 24)
page 2: 24 items (total 48)
page 3: 24 items (total 72)
page 4: 24 items (total 96)
page 5: 24 items (total 120)
page 6: 24 items (total 144)
page 7: 24 items (total 168)
page 8: 24 items (total 192)
page 9: 24 items (total 216)
page 10: 24 items (total 240)
page 11: 24 items (total 264)
page 12: 24 items (total 288)
page 13: 24 items (total 312)
page 14: 24 items (total 336)
page 15: 24 items (total 360)
page 16: 24 items (total 384)
page 17: 24 items (total 408)
page 18: 24 items (total 432)
page 19: 24 items (total 456)
page 20: 24 items (total 480)
page 21: 24 items (total 504)
page 22: 24 items (total 528)
page 23: 24 items (total 552)
page 24: 24 items (total 576)
page 25: 24 items (total 600)
page 26: 24 items (total 624)
page 27: 24 items (total 648)
page 28: 24 items (total 672)
page 29: 24 items (total 696)
page 30: 24 items (total 720)
page 31: 24 items (total 744)
page 32: 24 items (total 768)
page 33: 24 items (total 792)
page 34: 24 items (tota

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

webtoon_list = []
weekend_dic = {0:'토요일', 1:'일요일'}
url = 'https://comic.naver.com/webtoon?tab=sat'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

for weekend in range(2):
    # 페이지를 가장 아래까지 스크롤하는 방법

    interval = 1 # 1초에 한번씩 스크롤 내림

    # 현재 문서 높이를 가져와서 저장
    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

    # 해당 Xpath 정보를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
    # elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

    columns = ['weekend', 'name', 'author', 'rate']         # 웹툰이름, 웹툰작가, 웹툰평점, 정보 크롤링
    values = []                                             # 토요일 웹툰 리스트와 일요일 웹툰 리스트 정보를 담을 빈 리스트 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')        # 현재 브라우저의 page의 html 정보를 가져오기
    data_rows = soup.find_all('li', attrs={'class':'item'}) # 해당 조건에 맞는 정보들을 list로 반환

    for i, row in enumerate(data_rows):                     # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다

        print('{0} {1}번째 웹툰정보 크롤링'.format(weekend_dic[weekend], i+1))
        blank = []
        blank.append(weekend_dic[weekend])

        name = row.find('span', attrs={'class':'ContentTitle__title--e3qXt'})       # 웹툰이름 가져오기
        if name:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
            name = name.get_text().strip()                                          # html에서 텍스트만 가져온 뒤 공백 제거
            blank.append(name)
        else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
            blank.append('Something is wrong')
            print('{}번째 웹툰이름 가져올때 문제발생'.format(i+1))
            continue


        author = row.find('a', attrs={'class':'ContentAuthor__author--CTAAP'})      # 웹툰작가 가져오기
        if author:
            author = author.get_text().strip()
            blank.append(author)
        else:
            blank.append('Something is wrong')
            print('{}번째 웹툰작가 가져올때 문제발생'.format(i+1))
            continue


        rate = row.find('span', attrs={'class':'Rating__star_area--dFzsb'})        # 웹툰평점 가져오기
        if rate:
            rate = rate.find('span', attrs={'class':'text'}).get_text().strip()    # class가 Rating__star_area--dFzsb인 span 태그 안에 있는 span 태그에서 텍스트 가져오기
            blank.append(rate)
        else:
            blank.append('Something is wrong')
            print('{}번째 웹툰평점 가져올때 문제발생'.format(i+1))
            continue

        values.append(blank)
        print('---------------------------------------------------')

    sunday_webtoon_button = driver.find_element(By.XPATH, '//*[@id="wrap"]/header/div[3]/nav/ul/li[8]/a') # 일요일 웹툰 클릭
    sunday_webtoon_button.click()
    time.sleep(1)

    df = pd.DataFrame(values, columns = columns)
    webtoon_list.append(df) # 최종적으로 [[토요일 웹툰 정보 리스트들], [일요일 웹툰 정보 리스트들]] 형태로 반환됨

# 브라우저 종료
driver.quit()
